# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

### LangSmith Configuration

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - 13 Advanced Retrieval - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [6]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-02 17:13:03--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.02s   

2025-03-02 17:13:04 (1.02 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-03-02 17:13:04--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [8]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 2, 27, 17, 13, 6, 696577)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [9]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [10]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [12]:
from langchain_openai import ChatOpenAI
# Changed to model with larger token limit to prevent errors when invoking some retriever chains
chat_model = ChatOpenAI(model="gpt-4o-mini")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [13]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [14]:
# naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [15]:
# naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [16]:
# naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [17]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [18]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [19]:
# bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [20]:
# bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [21]:
# bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [22]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [23]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [24]:
# contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [25]:
# contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [26]:
# contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [28]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [29]:
# multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [30]:
# multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [31]:
# multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [32]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [33]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/tmp/ipykernel_288434/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [34]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [35]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [36]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [37]:
# parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [38]:
# parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [39]:
# parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [40]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [41]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [42]:
# ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [43]:
# ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [44]:
# ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [45]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [46]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [47]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [48]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [49]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [50]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [51]:
# semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

In [52]:
# semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

In [53]:
# semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

### Golden Dataset using SDG

In [55]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(chat_model)
generator_embeddings = LangchainEmbeddingsWrapper(embeddings)

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
golden_dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node b9c28294-d160-4313-9f74-05a91478669b does not have a summary. Skipping filtering.
Node b4b03266-ae0e-49ae-98b3-a6cd891c7fdc does not have a summary. Skipping filtering.
Node 9c17c0cf-ef3b-4942-b556-f76d79e90eda does not have a summary. Skipping filtering.
Node db9a4101-77ce-4e8f-8e09-9a3039a44502 does not have a summary. Skipping filtering.
Node b342e0c5-eae5-4c3d-b639-efa5fdc1b04c does not have a summary. Skipping filtering.
Node e2035e9b-1294-4303-b17d-d4c30530c1c7 does not have a summary. Skipping filtering.
Node 067b3482-3ebd-4d95-80ec-5877e449a2f2 does not have a summary. Skipping filtering.
Node 7d8cd4bc-c22d-4b9d-a57f-f75755ef2c78 does not have a summary. Skipping filtering.
Node 2e4605d2-750e-4d0b-a055-04a017b42094 does not have a summary. Skipping filtering.
Node 52ef3592-3a39-4492-acfd-a8b0b3ce638b does not have a summary. Skipping filtering.
Node 111cf59a-c84b-47d5-a32d-d99ae7820501 does not have a summary. Skipping filtering.
Node 3a426d88-cd7a-4eda-af72-f4f95823489a d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/244 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Bad request: invalid \'dotted_order\': dotted_order ac875ea7-f2f4-4a32-8bce-5ff65417ef53 has timestamp 2025-03-02 23:15:25.69464 +0000 UTC earlier than parent timestamp 2025-03-02 23:15:25.989045 +0000 UTC for run_id:ac875ea7-f2f4-4a32-8bce-5ff65417ef53 trace_id:9083ffa8-eeb6-4d8d-aaaa-fa22875269b5 dotted_order:20250302T231525989045Z9083ffa8-eeb6-4d8d-aaaa-fa22875269b5.20250302T231525694640Zac875ea7-f2f4-4a32-8bce-5ff65417ef53 parent_run_id:9083ffa8-eeb6-4d8d-aaaa-fa22875269b5"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/mul

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Bad request: invalid \'dotted_order\': dotted_order ac875ea7-f2f4-4a32-8bce-5ff65417ef53 has timestamp 2025-03-02 23:15:25.69464 +0000 UTC earlier than parent timestamp 2025-03-02 23:15:25.989045 +0000 UTC for run_id:ac875ea7-f2f4-4a32-8bce-5ff65417ef53 trace_id:9083ffa8-eeb6-4d8d-aaaa-fa22875269b5 dotted_order:20250302T231525989045Z9083ffa8-eeb6-4d8d-aaaa-fa22875269b5.20250302T231525694640Zac875ea7-f2f4-4a32-8bce-5ff65417ef53 parent_run_id:9083ffa8-eeb6-4d8d-aaaa-fa22875269b5"}\n')


In [56]:
golden_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are the key elements that make Keanu Reev...,[: 0\nReview: The best way I can describe John...,Keanu Reeves' performance in John Wick stands ...,single_hop_specifc_query_synthesizer
1,What are the general public's opinions on the ...,[: 2\nReview: With the fourth installment scor...,The fourth installment of John Wick is scoring...,single_hop_specifc_query_synthesizer
2,What make Keanu special in John wick?,[: 3\nReview: John wick has a very simple reve...,Keanu is special in John Wick because he portr...,single_hop_specifc_query_synthesizer
3,How does the portrayal of Russian mobsters con...,[: 4\nReview: Though he no longer has a taste ...,"In the film, the portrayal of Russian mobsters...",single_hop_specifc_query_synthesizer
4,What motivates John Wick in the film?,[: 5\nReview: Ultra-violent first entry with l...,"In the original John Wick (2014), the ex-hit-m...",single_hop_specifc_query_synthesizer
5,How does the visual style of 'John Wick: Chapt...,[<1-hop>\n\n: 8\nReview: About 6 months ago I ...,'John Wick: Chapter 3 - Parabellum' is noted f...,multi_hop_specific_query_synthesizer
6,In what ways does John Wick Chapter 2 maintain...,[<1-hop>\n\n: 23\nReview: I love me a bit of t...,John Wick Chapter 2 maintains its entertainmen...,multi_hop_specific_query_synthesizer
7,How does the action and plot of 'John Wick Cha...,[<1-hop>\n\n: 20\nReview: In a world where mov...,'John Wick Chapter 2' is described as relentle...,multi_hop_specific_query_synthesizer
8,Why was John Wick 3 a disappointment despite b...,"[<1-hop>\n\n: 16\nReview: Ok, so I got back fr...",John Wick 3 was a disappointment for the revie...,multi_hop_specific_query_synthesizer
9,How does Keanu Reeves' portrayal of John Wick ...,[<1-hop>\n\n: 20\nReview: John Wick is somethi...,Keanu Reeves' portrayal of John Wick significa...,multi_hop_specific_query_synthesizer


### RAGAS Evaluation

In [57]:
from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import context_precision, context_recall, context_entity_recall, answer_relevancy, faithfulness
import pandas as pd
import json

retriever_specific_metrics = [context_precision, context_recall, context_entity_recall]
additional_metrics = [faithfulness, answer_relevancy]
all_metrics = retriever_specific_metrics + additional_metrics

def run_ragas_evaluation(retriever_chain, dataset):
    for test_row in dataset:
        response = retriever_chain.invoke({"question" : test_row.eval_sample.user_input})
        test_row.eval_sample.response = response["response"].content
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]] 

    evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
    evaluator_llm = LangchainLLMWrapper(chat_model)

    custom_run_config = RunConfig(timeout=360)

    result = evaluate(
        dataset=evaluation_dataset,
        metrics=all_metrics,
        llm=evaluator_llm,
        run_config=custom_run_config
    )

    return result

def add_results_to_df(results_df, name, result):
    df = pd.DataFrame([{"retriver_name": name}])
    df = pd.concat([df, pd.DataFrame(json.loads(f"{result}".replace("'", '"')), index=[0])], axis=1)
    if results_df is None:
        results_df = df
    else:
        results_df = pd.concat([results_df, df], ignore_index=True)
    return results_df

In [58]:
retriever_chains = {
    "Naive": naive_retrieval_chain,
    "BM25": bm25_retrieval_chain,
    "Contextual Compression": contextual_compression_retrieval_chain,
    "Parent Document": parent_document_retrieval_chain,
    "Multi-Query": multi_query_retrieval_chain,
    "Ensemble": ensemble_retrieval_chain,
    # "Semantic Chunking": semantic_retrieval_chain
}

results_df = None
for name, retriever_chain in retriever_chains.items():
    print(f"Evaluating {name}")
    result = run_ragas_evaluation(retriever_chain.with_config({"run_name": name}), golden_dataset)
    print(result)
    results_df = add_results_to_df(results_df, name, result)
    print("\n" + "="*50 + "\n")

results_df

Evaluating Naive


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'context_precision': 0.5863, 'context_recall': 0.7500, 'context_entity_recall': 0.4900, 'faithfulness': 0.8509, 'answer_relevancy': 0.8669}


Evaluating BM25


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'context_precision': 0.3806, 'context_recall': 0.6667, 'context_entity_recall': 0.5683, 'faithfulness': 0.6464, 'answer_relevancy': 0.6760}


Evaluating Contextual Compression


...

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'context_precision': 0.6520, 'context_recall': 0.9000, 'context_entity_recall': 0.4650, 'faithfulness': 0.9411, 'answer_relevancy': 0.8627}


Evaluating Ensemble


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'context_precision': 0.6336, 'context_recall': 0.8333, 'context_entity_recall': 0.5617, 'faithfulness': 0.8978, 'answer_relevancy': 0.8750}




,retriver_name,context_precision,context_recall,context_entity_recall,faithfulness,answer_relevancy
0,Naive,0.5863,0.7500,0.4900,0.8509,0.8669
1,BM25,0.3806,0.6667,0.5683,0.6464,0.6760
2,Contextual Compression,0.6917,0.5333,0.4300,0.8402,0.6745
3,Parent Document,0.5583,0.4333,0.4467,0.5814,0.7687
4,Multi-Query,0.6520,0.9000,0.4650,0.9411,0.8627
5,Ensemble,0.6336,0.8333,0.5617,0.8978,0.8750


### Add Latency and Cost from LangSmith

In [59]:
retriever_stats = {
    "Naive": {"latency": 4.057, "cost": 0.00067311},
    "BM25": {"latency": 3.729, "cost": 0.0002965},
    "Contextual Compression": {"latency": 4.12, "cost": 0.000302025},
    "Parent Document": {"latency": 3.2175, "cost": 0.00020503125},
    "Multi-Query": {"latency": 7.186, "cost": 0.000837795},
    "Ensemble": {"latency": 7.18125, "cost": 0.00091528125},
}

for name, stats in retriever_stats.items():
    results_df.loc[(results_df['retriver_name'] == name), 'latency'] = stats['latency']
    results_df.loc[(results_df['retriver_name'] == name), 'cost'] = stats['cost']

results_df

,retriver_name,context_precision,context_recall,context_entity_recall,faithfulness,answer_relevancy,latency,cost
0,Naive,0.5863,0.7500,0.4900,0.8509,0.8669,4.05700,0.000673
1,BM25,0.3806,0.6667,0.5683,0.6464,0.6760,3.72900,0.000296
2,Contextual Compression,0.6917,0.5333,0.4300,0.8402,0.6745,4.12000,0.000302
3,Parent Document,0.5583,0.4333,0.4467,0.5814,0.7687,3.21750,0.000205
4,Multi-Query,0.6520,0.9000,0.4650,0.9411,0.8627,7.18600,0.000838
5,Ensemble,0.6336,0.8333,0.5617,0.8978,0.8750,7.18125,0.000915


# Best Retriever Analysis
Based on the *retriever-specific* evaluation metrics and operational considerations, the optimal retriever choice ***depends on specific use case requirements***. For *precision-focused* applications requiring minimal latency, **Contextual Compression** offers the best accuracy-quality balance while maintaining moderate costs. For *recall-critical* use cases like comprehensive question answering, **Multi-Query** achieves exceptional context recall and solid precision, though at higher latency and cost. *Budget-conscious* implementations should consider **Parent Document Retrieval** (lowest cost, fastest) despite its lower recall metrics. **BM25** provides a cost-effective middle ground suitable for *general-purpose* use. The **Ensemble** method balances multiple metrics well but incurs the highest operational costs, making it most appropriate for *mission-critical* applications where accuracy justifies the expense.